In [ ]:
%matplotlib inline


# Wind and Sea Level Pressure Interpolation

Interpolate sea level pressure, as well as wind component data,
to make a consistent looking analysis, featuring contours of pressure and wind barbs.

Adapted from: https://unidata.github.io/MetPy/latest/examples/gridding/Wind_SLP_Interpolation.html


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from metpy.calc import wind_components
from metpy.calc import reduce_point_density
from metpy.cbook import get_test_data
from metpy.interpolate import interpolate_to_grid, remove_nan_observations
from metpy.plots import add_metpy_logo, current_weather, sky_cover, StationPlot
from metpy.units import units
from metpy.io import metar

#to_proj = ccrs.AlbersEqualArea(central_longitude=-97., central_latitude=38.)
to_proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=35,
                             standard_parallels=[35])

Read in data



In [ ]:
## define the time we're plotting
datetime = pd.Timestamp(2021, 12, 11, 3)  ## year, month, day, hour, (minute, second)

In [ ]:
#data = metar.parse_metar_file('metar_20211211_0300.txt')
data = metar.parse_metar_file("metar_"+datetime.strftime("%Y%m%d_%H%M")+".txt")

# Drop rows with missing winds
data = data.dropna(how='any', subset=['wind_direction', 'wind_speed'])

Take a quick look at the data we're using:

In [ ]:
data

This METAR dataset includes surface data from around the world, but the interpolation method can sometimes fail when getting close to the poles, etc. So we'll subset the data down to the general region of North America. 

In [ ]:
## subset to -50 to -150 longitude and 15-70 latitude 
data = data[(data.longitude < -50) & (data.longitude > -150) & (data.latitude > 15) & (data.latitude < 70)]

This sample data has *way* too many stations to plot all of them. The number
of stations plotted will be reduced using `reduce_point_density`.

In [ ]:
# Set up the map projection
proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=35,
                             standard_parallels=[35])

# Use the Cartopy map projection to transform station locations to the map and
# then refine the number of stations plotted by setting a 150km radius
point_locs = proj.transform_points(ccrs.PlateCarree(), data['longitude'].values,
                                   data['latitude'].values)
data = data[reduce_point_density(point_locs, 200000.)]

Now move on to the interpolation. First, project the lon/lat locations to our final projection



In [ ]:
lon = data['longitude'].values
lat = data['latitude'].values
xp, yp, _ = to_proj.transform_points(ccrs.Geodetic(), lon, lat).T

Remove all missing data from pressure



In [ ]:
x_masked, y_masked, pressure = remove_nan_observations(xp, yp, data['air_pressure_at_sea_level'].values)

Interpolate pressure using Cressman interpolation



In [ ]:
slpgridx, slpgridy, slp = interpolate_to_grid(x_masked, y_masked, pressure,
                                              interp_type='cressman', minimum_neighbors=1,
                                              search_radius=400000, hres=100000)

Get wind information and mask where either speed or direction is unavailable



In [ ]:
wind_speed = (data['wind_speed'].values * units('m/s')).to('knots')
wind_dir = data['wind_direction'].values * units.degree

good_indices = np.where((~np.isnan(wind_dir)) & (~np.isnan(wind_speed)))

x_masked = xp[good_indices]
y_masked = yp[good_indices]
wind_speed = wind_speed[good_indices]
wind_dir = wind_dir[good_indices]

Calculate u and v components of wind and then interpolate both.

Both will have the same underlying grid so throw away grid returned from v interpolation.



In [ ]:
u, v = wind_components(wind_speed, wind_dir)

windgridx, windgridy, uwind = interpolate_to_grid(x_masked, y_masked, np.array(u),
                                                  interp_type='cressman', search_radius=400000,
                                                  hres=100000)

_, _, vwind = interpolate_to_grid(x_masked, y_masked, np.array(v), interp_type='cressman',
                                  search_radius=400000, hres=100000)

Get temperature information



In [ ]:
x_masked, y_masked, t = remove_nan_observations(xp, yp, data['air_temperature'].values)
tempx, tempy, temp = interpolate_to_grid(x_masked, y_masked, t, interp_type='cressman',
                                         minimum_neighbors=3, search_radius=400000, hres=35000)

temp = np.ma.masked_where(np.isnan(temp), temp)

Plot the surface map as before, but now plot the analyzed temperature and pressure fields on the map.



In [ ]:
# Change the DPI of the resulting figure. Higher DPI drastically improves the
# look of the text rendering.
plt.rcParams['savefig.dpi'] = 255

# Create the figure and an axes set to the projection.
fig = plt.figure(figsize=(20, 10))
#add_metpy_logo(fig, 1100, 300, size='large')
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Add some various map elements to the plot to make it recognizable.
#ax.add_feature(cfeature.LAND)
#ax.add_feature(cfeature.OCEAN)
#ax.add_feature(cfeature.LAKES)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)

# Set plot bounds
#ax.set_extent((-118, -73, 23, 50))
ax.set_extent((-108,-73, 23, 50))

#
# Here's the actual station plot
#

# Start the station plot by specifying the axes to draw on, as well as the
# lon/lat of the stations (with transform). We also the fontsize to 12 pt.
stationplot = StationPlot(ax, data['longitude'].values, data['latitude'].values,
                          clip_on=True, transform=ccrs.PlateCarree(), fontsize=10)

# Plot the temperature and dew point to the upper and lower left, respectively, of
# the center point. Each one uses a different color.
stationplot.plot_parameter('NW', data['air_temperature'].values, color='red')
stationplot.plot_parameter('SW', data['dew_point_temperature'].values,
                           color='darkgreen')

# A more complex example uses a custom formatter to control how the sea-level pressure
# values are plotted. This uses the standard trailing 3-digits of the pressure value
# in tenths of millibars.
stationplot.plot_parameter('NE', data['air_pressure_at_sea_level'].values,
                           formatter=lambda v: format(10 * v, '.0f')[-3:])

# Plot the cloud cover symbols in the center location. This uses the codes made above and
# uses the `sky_cover` mapper to convert these values to font codes for the
# weather symbol font.
stationplot.plot_symbol('C', data['cloud_coverage'].values, sky_cover)

# Same this time, but plot current weather to the left of center, using the
# `current_weather` mapper to convert symbols to the right glyphs.
stationplot.plot_symbol('W', data['current_wx1_symbol'].values, current_weather)

# Add wind barbs
stationplot.plot_barb(data['eastward_wind'].values, data['northward_wind'].values,
                     length=6)

# Also plot the actual text of the station id. Instead of cardinal directions,
# plot further out by specifying a location of 2 increments in x and 0 in y.
stationplot.plot_text((1.5, -1), data['station_id'].values, fontsize=8, color='dimgray')

###########
## now add analyzed MSLP contours
cs = ax.contour(slpgridx, slpgridy, slp, colors='k', levels=list(range(972, 1034, 4)))
ax.clabel(cs, inline=1, fontsize=12, fmt='%i')

## and 2-m temperature
cs2 = ax.contour(tempx, tempy, temp, colors='r', levels=list(range(-30,30,5)))
ax.clabel(cs2, inline=1, fontsize=12, fmt='%i', colors='r')

plt.title('METAR observations, MSLP, 2-m temperature', loc='left')
plt.title(datetime.strftime("%H%M UTC %d %b %Y"), loc='right')

plt.show()

In [ ]:
## save file:
fig.savefig("sfc_analysis_simple.png", bbox_inches='tight', facecolor='white', transparent=False)